*This notebook was adapted from [here](https://github.com/rmassei/2024-jn-omero-pipeline), with thanks to the original author Riccardo Massei.*

# Workflow from OMERO - Cell Nuclei Segmentation

The present jupyter notebook shows an automatic pipeline to fetch data from OMERO, perform several processing steps and push back the results OMERO adding potential metadata and Region of Interest (ROIs).

We suggest having OMERO web opened in a browser tab to follow the updates. Remember to click on the update button after new uploads:

![](../screenshots/omero_update_button.png)

Most functions are based on [ezomero](https://github.com/TheJacksonLaboratory/ezomero?tab=readme-ov-file#ezomero) and [scikit-image](https://scikit-image.org/). A few commands come from the OMERO Python bindings. Please refer to the documentation of these packages for the functions details:
- [ezomero API documentation](https://thejacksonlaboratory.github.io/ezomero/genindex.html)
- [scikit-image API documentation](https://scikit-image.org/docs/stable/api/api.html)
- [OMERO Python language bindings](https://omero.readthedocs.io/en/v5.6.9/developers/Python.html)

## 1) Import Packages

In [ ]:
import getpass

import ezomero
import pandas as pd
import tifffile
import numpy as np

from skimage import filters, morphology, measure
from skimage.color import label2rgb
from matplotlib import pyplot as plt

## 2) Set Parameters to login into OMERO

Add here your username or password. Adjust the OMERO host and port accordingly.

In [ ]:
OMEROUSER = input(f"Enter username: \t")
OMEROPASS = getpass.getpass(prompt = f"Enter password: \t")
OMEROGROUP = "workshop_2025"
OMEROHOST = "141.76.16.87"
OMEROPORT = 4064

## 3) Connect to OMERO

In [ ]:
# Connection Check:
conn=ezomero.connect(user=OMEROUSER, 
                     password=OMEROPASS, 
                     group=OMEROGROUP, 
                     host=OMEROHOST, 
                     port=OMEROPORT, 
                     secure=True)

In [ ]:
## Information about the connection and its status
print(conn.isConnected())
user = conn.getUser()
print("Current user:")
print("   ID:", user.getId())
print("   Username:", user.getName())
print("   Full Name:", user.getFullName())

## 4) Browse your OMERO project folder

Browse your project and have an overview on the project and their content

In [ ]:
proj_ids = ezomero.get_project_ids(conn)
print("Current Project IDs: {proj_ids}".format(proj_ids=proj_ids))

In [ ]:
project_id = 

In [ ]:
ds_ids = ezomero.get_dataset_ids(conn, project=project_id)
print("Current Dataset IDs: {ds_ids}".format(ds_ids=ds_ids))

In [ ]:
dataset_id = 

In [ ]:
img_ids = ezomero.get_image_ids(conn, dataset=dataset_id)
print("Current Image IDs: {img_ids}".format(img_ids=img_ids))

In [ ]:
img_id = 

### 4.1) Select your image

Read image object and image pixel data. Check the [ezomero.get_image](https://thejacksonlaboratory.github.io/ezomero/ezomero.html#ezomero.get_image) documentation.

In [ ]:
img_obj, image = ezomero.get_image() # fill in the necessary arguments

### 4.2) Get information about your image

In [ ]:
print("Image name: ", img_obj.getName(), img_obj.getDescription())
# Retrieve information about an image.
print("Image shape (TZYXC):", image.shape)
# List Channels (loads the Rendering settings to get channel colors)
for channel in img_obj.getChannels():
    print ('Channel:', channel.getLabel(),)
    print ('Color:', channel.getColor().getRGB())
    print ('Lookup table:', channel.getLut())

### 4.3) Plot your images

In [ ]:
# Remove unitary dimensions
nuclei = np.squeeze(image)
print("Image shape:", nuclei.shape)

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(10,10))
axs.imshow(nuclei, cmap='gray')
axs.set_title(f'{img_obj.getName()}')

## 5) Image Processing 1 - Denoising

Apply a gaussian filter to denoise

In [ ]:
nuclei_gaussian = filters.gaussian(image=nuclei,
                                   sigma=2)

fig, axs = plt.subplots(1,2, figsize=(10,10))
axs[0].imshow(nuclei, cmap='gray')
axs[0].set_title('Nuclei - Original')
axs[1].imshow(nuclei_gaussian, cmap='gray')
axs[1].set_title('Nuclei - Processed')

## 6) Image Processing 2 - Binarization and Labelling

Apply a OTSU threshold and create the labelling

In [ ]:
# Create a disk with radius = 3
disk = morphology.disk(radius=3)

# Apply Otsu thresholding
threshold = filters.threshold_otsu(nuclei_gaussian)
binary_image = nuclei_gaussian >= threshold

# Apply binary opening
opened = morphology.binary_opening(image=binary_image,
                                    footprint=disk)

# Label nuclei
labeled_nuclei = measure.label(opened)

# Remove small objects
cleaned = morphology.remove_small_objects(ar=labeled_nuclei, 
                                          min_size=50)

# Create overlay
image_label_overlay = label2rgb(label=cleaned,
                                image=nuclei,
                                bg_label=0)

fig, axs = plt.subplots(1,3, figsize=(15,10))
axs[0].imshow(nuclei_gaussian, cmap='gray')
axs[0].set_title('Original')
axs[1].imshow(binary_image, cmap='gray')
axs[1].set_title('Binary')
axs[2].imshow(image_label_overlay)
axs[2].set_title('Labels')

Save the labelled image locally as a TIFF file (optional)

In [ ]:
tifffile.imwrite('test.tif', 
                 data=cleaned, 
                 dtype=cleaned.dtype, 
                 shape=cleaned.shape)

## 7) Extract table with features

In [ ]:
properties = ['label', 'area', 'mean_intensity', 'major_axis_length', 'minor_axis_length']
features = measure.regionprops_table(label_image=cleaned,
                                       intensity_image=nuclei,
                                       properties=properties)
df = pd.DataFrame(features)
df

Save table locally as a CSV file (optional)

In [ ]:
df.to_csv("feature_extraction.csv")

## 8) Push back the results to OMERO and add metadata plus other information

Make image 5D for importing into OMERO

In [ ]:
labelled_nuclei = cleaned[np.newaxis, np.newaxis, :, :, np.newaxis] # tzyxc
print("Labels new shape (TZYXC):", labelled_nuclei.shape)

Create a new dataset. Check the [ezomero.post_dataset](https://thejacksonlaboratory.github.io/ezomero/ezomero.html#ezomero.post_dataset) documentation.

In [ ]:
## Create a new dataset
new_dataset_id = ezomero.post_dataset() # fill in the necessary arguments

Create a new image with the labelled nuclei data in the new dataset. Check the [ezomero.post_image](https://thejacksonlaboratory.github.io/ezomero/ezomero.html#ezomero.post_image) documentation. pay attention to the `dim_order` parameter, it must match the shape of your image.

In [ ]:
# Push image to new dataset 
new_image_id = ezomero.post_image() # fill in the necessary arguments

### 8.1) Add key-value pairs as annotations

Key values pairs can be added to the image as annotations from a python dictionary. We created a `kv_pairs` dictionary below. Post these annotations to the processed image using the [ezomero.post_map_annotation](https://thejacksonlaboratory.github.io/ezomero/ezomero.html#ezomero.post_map_annotation) function.

In [ ]:
kv_pairs = {
    "Analysis Result Type" : "Processed",
    "Data Used for The Analysis": f"{img_obj.getName()}",
    "Analysis method and details" : "Skimage"
}

map_ann_id = ezomero.post_map_annotation() # fill in the necessary arguments

### 8.2) Add description and table with the results

Add a description to the image using the [ezomero.put_description](https://thejacksonlaboratory.github.io/ezomero/ezomero.html#ezomero.put_description) function.

In [ ]:
# Update existing description
ezomero.put_description(conn,
                        obj_type='Image',
                        obj_id=new_image_id,
                        desc="This Image Was Created")

Add table results using the [ezomero.post_table](https://thejacksonlaboratory.github.io/ezomero/ezomero.html#ezomero.post_table) function.

In [ ]:
# Post table
table_id = ezomero.post_table(conn, 
                              table=df, 
                              object_type="Image",
                              object_id=new_image_id,
                              title='Features Results',
                              headers=True) # this uploads the file, but it cannot be viewed or downloaded properly in OMERO Insight, only in OMERO web with omero table

## 9) Close Connection

Run the cell below to properly close the connection to OMERO. If you need to re-open it, connect again by running the [3) Connect to OMERO](#3_connect_to_omero)-cell.

In [ ]:
conn.close()

## 10) OPTIONAL - Add masks as ROIs

Add the labelling directly as ROIs in OMERO with ezomero. 

Some useful functions are:
- [skimage.measure.regionprops](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops), particularly the `coords` attribute to get the pixel coordinates of each labelled region
- [ezomero.rois.Polygon](https://thejacksonlaboratory.github.io/ezomero/ezomero.html#ezomero.rois.Polygon)
- [ezomero.post_roi](https://thejacksonlaboratory.github.io/ezomero/ezomero.html#ezomero.post_roi)

*Advanced: use the glasbey colormap from the [cmap](https://cmap-docs.readthedocs.io/en/stable/colors/#usage) library below to color each ROI differently*

<details>
  <summary>Click here to see a more details for a possible implementation</summary>

- Get each labelled region coordinates from the labelled image using `coords` from [skimage.measure.regionprops](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops)
- Make a list of [ezomero.rois.Polygon](https://thejacksonlaboratory.github.io/ezomero/ezomero.html#ezomero.rois.Polygon)
- Use ezomero to post the list of polygons as ROIs to the original image

</details>

In [ ]:
# from cmap import Colormap
# glasbey_cm = Colormap('glasbey:glasbey') 

# Automatize the pipeline trought a dataset on OMERO

By getting the image ids, one can easily make a **for** loop in python to run the workflow across all images in a dataset.

In [ ]:
dataset = conn.getObject("Dataset", dataset_id)

ID =[]

for images in dataset.listChildren():
    ID.append(images.getId())

print(ID)

for id in ID:
    img = conn.getObject("Image", id)
    ### repeat the pipeline ###